In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import scipy
from sklearn.metrics import classification_report,accuracy_score
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
from pylab import rcParams
rcParams['figure.figsize'] = 14, 8
RANDOM_SEED = 42
from datetime import datetime
import seaborn as sns

In [2]:
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import NearMiss

In [3]:
db=pd.read_csv('ml_labelled_Data.csv')

In [4]:
df=db[['Day_of_Week' , 'Weather_Conditions' , 'Road_Surface_Conditions'
                          , 'Light_Conditions','Speed_limit','Junction_Detail','Road_Type','Accident_Severity']]
df.head()

,Day_of_Week,Weather_Conditions,Road_Surface_Conditions,Light_Conditions,Speed_limit,Junction_Detail,Road_Type,Accident_Severity
0,5,6,5,4,30,4,3,1
1,6,2,1,1,30,0,0,2
2,4,2,1,1,30,4,3,2
3,0,2,1,4,30,4,3,2
4,1,2,5,0,30,4,3,2


In [5]:
df['Accident_Severity'].value_counts(
)

2    895883
1    138192
0     14500
Name: Accident_Severity, dtype: int64

In [17]:
df2 = df[df['Accident_Severity'] == 2]

In [18]:
df2 = df2.iloc[0:200000,:]
df2['Accident_Severity'].value_counts()

2    200000
Name: Accident_Severity, dtype: int64

In [19]:
df1 = df[df['Accident_Severity'] == 1]

In [20]:
df0 = df[df['Accident_Severity'] == 0]

In [24]:
df_target=pd.concat([df2,df1,df0])
df_target['Accident_Severity'].value_counts()

2    200000
1    138192
0     14500
Name: Accident_Severity, dtype: int64

In [25]:
df.drop(['Accident_Severity'] ,axis = 1)

,Day_of_Week,Weather_Conditions,Road_Surface_Conditions,Light_Conditions,Speed_limit,Junction_Detail,Road_Type
0,5,6,5,4,30,4,3
1,6,2,1,1,30,0,0
2,4,2,1,1,30,4,3
3,0,2,1,4,30,4,3
4,1,2,5,0,30,4,3
...,...,...,...,...,...,...,...
1048570,4,2,1,4,60,4,3
1048571,3,2,3,3,60,4,3
1048572,5,2,3,4,30,9,3
1048573,5,2,5,3,60,9,3


In [27]:
df_final = pd.concat([df,df_target])

In [28]:
#Create independent and Dependent Features
columns = df_final.columns.tolist()
# Filter the columns to remove data we do not want 
columns = [c for c in columns if c not in ["Accident_Severity"]]
# Store the variable we are predicting 
target = "Accident_Severity"
# Define a random state 
state = np.random.RandomState(42)
X = df_final[columns]
Y = df_final[target]
# Print the shapes of X & Y
print(X.shape)
print(Y.shape)

(1401267, 7)
(1401267,)


In [29]:
nm = NearMiss()
X_res,y_res=nm.fit_resample(X,Y)

In [30]:
X_res.shape,y_res.shape

((87000, 7), (87000,))

In [31]:
import sklearn.model_selection as model_selection
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_res.values, 
                                              y_res.values,test_size=0.20, random_state=99)

In [32]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
lr = LogisticRegression()
# Fit the model on the trainng data.
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
sk_report = classification_report(
    digits=6,
    y_true=y_test, 
    y_pred=y_pred)
print("Accuracy", round(accuracy_score(y_pred, y_test)*100,2))
print(sk_report)
pd.crosstab(y_test, y_pred, rownames=['Actual'], colnames=['Predicted'], margins=True)

Accuracy 51.43
              precision    recall  f1-score   support

           0   0.602682  0.594093  0.598357      5824
           1   0.347095  0.116510  0.174459      5845
           2   0.495823  0.838946  0.623282      5731

    accuracy                       0.514310     17400
   macro avg   0.481867  0.516516  0.465366     17400
weighted avg   0.481630  0.514310  0.464171     17400



C:\Users\polis\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Predicted,0,1,2,All
Actual,,,,
0,3460,782,1582,5824
1,1857,681,3307,5845
2,424,499,4808,5731
All,5741,1962,9697,17400


In [33]:
# Implementing Oversampling for Handling Imbalanced 
smk = SMOTETomek(random_state=42)
X_res,y_res=smk.fit_resample(X,Y)

In [34]:
X_res.head()

,Day_of_Week,Weather_Conditions,Road_Surface_Conditions,Light_Conditions,Speed_limit,Junction_Detail,Road_Type
0,5,6,5,4,30,4,3
1,6,2,1,1,30,0,0
2,4,2,1,1,30,4,3
3,0,2,1,4,30,4,3
4,1,2,5,0,30,4,3


In [35]:
y_res.head()

0    1
1    2
2    2
3    2
4    2
Name: Accident_Severity, dtype: int64

In [36]:
d_data = pd.concat([X_res,y_res],axis=1)
d_data.head()

,Day_of_Week,Weather_Conditions,Road_Surface_Conditions,Light_Conditions,Speed_limit,Junction_Detail,Road_Type,Accident_Severity
0,5,6,5,4,30,4,3,1
1,6,2,1,1,30,0,0,2
2,4,2,1,1,30,4,3,2
3,0,2,1,4,30,4,3,2
4,1,2,5,0,30,4,3,2


In [37]:
d_data.to_csv('Synthetic_data.csv')

In [38]:
X_res.shape,y_res.shape

((3287645, 7), (3287645,))

In [39]:
from collections import Counter
print('Original dataset shape {}'.format(Counter(Y)))
print('Resampled dataset shape {}'.format(Counter(y_res)))

Original dataset shape Counter({2: 1095883, 1: 276384, 0: 29000})
Resampled dataset shape Counter({0: 1095883, 1: 1095881, 2: 1095881})


In [40]:
## RandomOverSampler to handle imbalanced data

from imblearn.over_sampling import RandomOverSampler

In [43]:
os =  RandomOverSampler()

In [45]:
X_train_res, y_train_res = os.fit_resample(X, Y)

In [46]:
X_train_res.shape,y_train_res.shape

((3287649, 7), (3287649,))

# Logistic Regression

In [47]:
import sklearn.model_selection as model_selection
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_res.values, 
                                              y_res.values,test_size=0.20, random_state=99)

In [49]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
lr = LogisticRegression()
# Fit the model on the trainng data.
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
sk_report = classification_report(
    digits=6,
    y_true=y_test, 
    y_pred=y_pred)
print("Accuracy", round(accuracy_score(y_pred, y_test)*100,2))
print(sk_report)
pd.crosstab(y_test, y_pred, rownames=['Actual'], colnames=['Predicted'], margins=True)

C:\Users\polis\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy 45.09
              precision    recall  f1-score   support

           0   0.497208  0.596018  0.542147    219017
           1   0.366582  0.181658  0.242932    219225
           2   0.440438  0.575137  0.498855    219287

    accuracy                       0.450903    657529
   macro avg   0.434743  0.450937  0.427978    657529
weighted avg   0.434724  0.450903  0.427949    657529



Predicted,0,1,2,All
Actual,,,,
0,130538,32845,55634,219017
1,74804,39824,104597,219225
2,57200,35967,126120,219287
All,262542,108636,286351,657529


# Navie Bayes

In [50]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
clf = GaussianNB()
clf.fit(X_train, y_train)
prediction = clf.predict(X_test)
sk_report = classification_report(
    digits=6,
    y_true=y_test, 
    y_pred=prediction)
print("Accuracy", round(accuracy_score(y_pred, y_test)*100,2))
print(sk_report)
pd.crosstab(y_test, y_pred, rownames=['Actual'], colnames=['Predicted'], margins=True)

Accuracy 45.09
              precision    recall  f1-score   support

           0   0.491283  0.550916  0.519393    219017
           1   0.368967  0.367860  0.368412    219225
           2   0.448966  0.395883  0.420757    219287

    accuracy                       0.438180    657529
   macro avg   0.436405  0.438220  0.436187    657529
weighted avg   0.436389  0.438180  0.436160    657529



Predicted,0,1,2,All
Actual,,,,
0,130538,32845,55634,219017
1,74804,39824,104597,219225
2,57200,35967,126120,219287
All,262542,108636,286351,657529


# Decision Tree

In [51]:
from sklearn.tree import DecisionTreeClassifier

In [52]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, y_train)
Y_pred = decision_tree.predict(X_test)
acc_decision_tree1 = round(decision_tree.score(X_test, y_test) * 100, 2)
sk_report = classification_report(
    digits=6,
    y_true=y_test, 
    y_pred=Y_pred)
print("Accuracy", acc_decision_tree1)
print(sk_report)
### Confusion Matrix 
pd.crosstab(y_test, Y_pred, rownames=['Actual'], colnames=['Predicted'], margins=True)

Accuracy 50.7
              precision    recall  f1-score   support

           0   0.547009  0.704580  0.615876    219017
           1   0.455161  0.231059  0.306517    219225
           2   0.486068  0.585475  0.531160    219287

    accuracy                       0.506983    657529
   macro avg   0.496079  0.507038  0.484518    657529
weighted avg   0.496062  0.506983  0.484481    657529



Predicted,0,1,2,All
Actual,,,,
0,154315,23084,41618,219017
1,74442,50654,94129,219225
2,53350,37550,128387,219287
All,282107,111288,264134,657529


# KNN

In [53]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=915)
neigh.fit(X_train, y_train)
y_pred = neigh.predict(X_test)

C:\Users\polis\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [54]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

prediction = neigh.predict(X_test)
sk_report = classification_report(
    digits=6,
    y_true=y_test, 
    y_pred=prediction)
print("Accuracy", round(accuracy_score(y_pred, y_test)*100,2))
print(sk_report)
pd.crosstab(y_test, y_pred, rownames=['Actual'], colnames=['Predicted'], margins=True)

C:\Users\polis\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


Accuracy 47.63
              precision    recall  f1-score   support

           0   0.513174  0.667496  0.580250    219017
           1   0.402213  0.222397  0.286422    219225
           2   0.470227  0.539156  0.502338    219287

    accuracy                       0.476295    657529
   macro avg   0.461871  0.476350  0.456336    657529
weighted avg   0.461856  0.476295  0.456302    657529



Predicted,0,1,2,All
Actual,,,,
0,146193,29855,42969,219017
1,80237,48755,90233,219225
2,58450,42607,118230,219287
All,284880,121217,251432,657529


In [56]:
from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier(n_estimators=200)
random_forest.fit(X_train,y_train)
Y_pred = random_forest.predict(X_test)
random_forest.score(X_test, y_test)
acc_random_forest1 = round(random_forest.score(X_test, y_test) * 100, 2)

sk_report = classification_report(
    digits=6,
    y_true=y_test, 
    y_pred=Y_pred)
print("Accuracy" , acc_random_forest1)
print(sk_report)
pd.crosstab(y_test, Y_pred, rownames=['Actual'], colnames=['Predicted'], margins=True)

Accuracy 50.72
              precision    recall  f1-score   support

           0   0.547337  0.704306  0.615979    219017
           1   0.451007  0.244032  0.316702    219225
           2   0.489155  0.573463  0.527965    219287

    accuracy                       0.507211    657529
   macro avg   0.495833  0.507267  0.486882    657529
weighted avg   0.495816  0.507211  0.486845    657529



Predicted,0,1,2,All
Actual,,,,
0,154255,24773,39989,219017
1,74387,53498,91340,219225
2,53186,40348,125753,219287
All,281828,118619,257082,657529
